# Description

This is the notebook for the creation of the first review network and derived hypotheses. 

* Using intermediary variables from workflow objects. In this workflow variables are directly used for the next step. 


* Review network: From Monarch knowledge graph, we built a network seeded by 8 nodes, retrieving their explicit relationships and all the relationships among all these nodes. Seed nodes:

    - 'MONDO:0007739' HD
    - 'HGNC:4851' Htt
    - 'CHEBI:18248' Iron
    - 'HGNC:18229' Rhes (RASD2)
    
Possible seed nodes:
https://monarchinitiative.org/search/Iron
* Connecting paths: query templates.

In [1]:
import transcriptomics, regulation, curation, monarch, graph, neo4jlib, hypothesis, summary, utils
import pandas as pd

## Import curated graph:

In [2]:
%%time
# initiate empty curation network:
curation_edges = pd.read_csv("curation/data/HD/HD_curated_edges.csv")
curation_nodes = pd.read_csv("curation/data/HD/HD_curated_nodes.csv")


CPU times: user 4.7 ms, sys: 521 µs, total: 5.22 ms
Wall time: 4.66 ms


## Construct the transcriptomics subgraph

In [3]:
%%time
# transcriptomics:
csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path, sep='\t')
clean_data = transcriptomics.clean_data(data)
data_edges = transcriptomics.prepare_data_edges(clean_data)
rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
rna_edges = transcriptomics.build_edges(rna_network)
rna_nodes, node_dict = transcriptomics.build_nodes(rna_network)
rna_edges = transcriptomics.rework_edges(pd.DataFrame(rna_edges), node_dict)


The function "read_data()" is running...

* This is the size of the raw expression data structure: (28087, 10)
* These are the expression attributes: Index(['Unnamed: 0', 'symbol', 'baseMean', 'HD.mean', 'Control.mean',
       'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj'],
      dtype='object')
* This is the first record:
           Unnamed: 0 symbol  baseMean    HD.mean  Control.mean  \
0  ENSG00000069011.10  PITX1  5.645675  18.684286      0.323793   

   log2FoldChange     lfcSE      stat        pvalue          padj  
0        4.769658  0.366367  13.01879  9.567529e-39  2.687232e-34  

The raw data is saved at: /home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/transcriptomics/HD/data/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.csv


Finished read_data().


The function "clean_data()" is running. Keeping only data with FC > 1.5 and FDR < 5% ...

* This is the size of the clean expression data structure: (3209, 6)
* These are the clean expressi

## Construct the regulation, Transcription factor targets subgraph

In [4]:
%%time
# regulation:
gmt_path = './regulation/msigdb/data/c3.tft.v6.1.entrez.gmt'
regulation.prepare_msigdb_data(gmt_path)

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
data_edges = regulation.prepare_data_edges(data, dicts)

# prepare regulation network
reg_network = regulation.prepare_regulation_edges(data_edges)

# build network with graph schema
reg_edges = regulation.build_edges(reg_network)
reg_nodes = regulation.build_nodes(reg_network)


The function "prepare_msigdb_data()" is running...

* Number of Transcription Factor Targets (TFT) gene sets: 615

The MSigDB raw network is saved at: /home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/regulation/msigdb/out/tf_genelist_entrez_msigdb.json. Other reporting files are also saved at the same directory.


Finished prepare_msigdb_data().


The function "load_tf_gene_edges()" is running...

Finished load_tf_gene_edges().


The function "get_gene_id_normalization_dictionaries()" is running...

* Querying BioThings to map gene symbols to HGNC and Entrez IDs...
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-3071...done.
Finished.
49 input query terms found no hit:
	['MEIS1BHOXA9', 'MYCMAX', 'E2F1DP2', 'COMP1', 'RORA1', 'INSAF', 'CACCCBINDINGFACTOR', 'CEBPGAMMA', '
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Saving not found gene symbols at: /home/karolis/LUMC/HDSR/bioknowledge-

# Construct the Monarch subgraph.

This takes a very long time!

If you have a file ready, better use the next cell!

In [5]:
%%time
# Monarch extraction:
# seed nodes
seedList = [ 
    'MONDO:0007739', # HD
    'HGNC:4851', # Htt
    'HGNC:182293', # Rhes
    'REACT:R-HSA-917937' #Iron uptake pathway
] 

# get first degree neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print("Nodes 1st degree neighbours:")
print(len(neighboursList))
# introduce animal model orthologs for seed nodes, can count as the 1st shell
seed_orthophenoList = monarch.get_orthopheno_list(seedList)
print(len(seed_orthophenoList))
## For 2nd shell nodes, or specifically, the neigbours of the seed nodes and captured orthologs in last query:
neighbours_orthophenoList = monarch.get_orthopheno_list(neighboursList)
print(len(neighbours_orthophenoList))

node_list = sum([seedList, neighboursList, seed_orthophenoList, neighbours_orthophenoList], 
               [])
monarch_network = monarch.extract_edges(node_list)
print("connections after extraction:")
print(len(monarch_network))

monarch.print_network(monarch_network, 'monarch_connections')


# build network with graph schema 
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)

# print type of objects
print('type edges:', type(monarch_edges))
print('type nodes:', type(monarch_nodes))
print()

# print objects sizes
print('len edges:', len(monarch_edges))
print('len nodes:', len(monarch_nodes))
print()

# print object attribute
print('attribute edges:', monarch_edges[0].keys())
print('attribute nodes:', monarch_nodes[0].keys())

# load networks and calculate graph nodes
graph_nodes_list, reg_graph_edges = graph.graph_nodes(
    curation=curation_edges,
    monarch=monarch_edges,
    transcriptomics=rna_edges,
    regulation=reg_edges
)

# Monarch graph connectivity
## get Monarch edges
monarch_network_graph = monarch.extract_edges(graph_nodes_list)
print('network: ',len(monarch_network_graph))

## save Monarch network
monarch.print_network(monarch_network_graph, 'monarch_connections_graph')
## build Monarch network with graph schema
monarch_graph_edges = monarch.build_edges(monarch_network_graph)
monarch_graph_nodes = monarch.build_nodes(monarch_network_graph)

  0%|          | 0/4 [00:00<?, ?it/s]


The function "get_neighbours_list()" is running. Its runtime may take some minutes. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


  0%|          | 0/4 [00:00<?, ?it/s]


Finished get_neighbours_list().

Nodes 1st degree neighbours:
1034

The function "get_orthopheno_list()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


  0%|          | 0/1034 [00:00<?, ?it/s]

...

Saving intermediary results in temp_keep and temp_finished, set rerun=True to restart api query from latest backup.


100%|█████████▉| 13380/13395 [12:20:16<01:05,  4.35s/it]

Saving intermediary results in temp_keep and temp_finished, set rerun=True to restart api query from latest backup.


100%|██████████| 13395/13395 [12:20:54<00:00,  3.32s/it]



Finished extract_edges(). To save the retrieved Monarch edges use the function "print_network()".

connections after 2nd degree:
207835

Saving Monarch edges at: '/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_connections_v2022-07-24.csv'...


The function "build_edges()" is running...
df (207835, 9)

* This is the size of the edges file data structure: (207835, 9)
* These are the edges attributes: Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')
* This is the first record:
                   subject_id                   object_id    property_id  \
0  ENSEMBL:ENSSSCG00000038466  ENSEMBL:ENSCAFG00000047620  RO:HOM0000017   

                   property_label property_description  \
0  in orthology relationship with                   NA   

                                   property_uri refere

## Import an already constructed monarch subgraph
Skip this cell if you generated your own subgraph!

In [5]:
monarch_edges = pd.read_csv("/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_edges_v2022-07-24.csv")
monarch_nodes = pd.read_csv("/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_nodes_v2022-07-24.csv")
monarch_edges = monarch_edges.to_dict('records')
monarch_nodes = monarch_nodes.to_dict('records')

In [6]:
# print type of objects
print('type edges:', type(monarch_edges))
print('type nodes:', type(monarch_nodes))
print()

# print objects sizes
print('len edges:', len(monarch_edges))
print('len nodes:', len(monarch_nodes))
print()

# print object attribute
print('attribute edges:', monarch_edges[0].keys())
print('attribute nodes:', monarch_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 207835
len nodes: 13394

attribute edges: dict_keys(['subject_id', 'property_id', 'object_id', 'reference_uri', 'reference_supporting_text', 'reference_date', 'property_label', 'property_description', 'property_uri'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'synonyms', 'description', 'name'])


## Merge all subgraphs into a Structured Review
Using the files generated from the previous cells are more stable than using the output variables.

In [7]:

# build review graph
edges = graph.build_edges(
    curation="curation/data/HD/HD_curated_edges.csv",
    monarch="/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_edges_v2022-07-24.csv",
    transcriptomics="/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/graph/rna_edges_v2022-07-25.csv",
    regulation="/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/graph/regulation_edges_v2022-07-25.csv"
    , input_from_file=True
)
nodes = graph.build_nodes(
    statements=edges,
    curation="curation/data/HD/HD_curated_nodes.csv",
    monarch="/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_nodes_v2022-07-24.csv",
    transcriptomics="/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/graph/rna_nodes_v2022-07-25.csv",
    regulation="/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/graph/regulation_nodes_v2022-07-25.csv",
    input_from_file=True

)


The function "build_edges()" is running...

Preparing networks...
Curated:
(1, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Monarch:
(207835, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Transcriptomics:
(3209, 9)
Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')
Regulatory:
(197267, 9)
Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')

Concatenatin

In [8]:
# print type of objects
print('type edges:', type(edges))
print('type nodes:', type(nodes))
print()

# print objects sizes
print('len edges:', len(edges))
print('len nodes:', len(nodes))
print()

# print object attribute
print('attribute edges:', edges.columns)
print('attribute nodes:', nodes.columns)

type edges: <class 'pandas.core.frame.DataFrame'>
type nodes: <class 'pandas.core.frame.DataFrame'>

len edges: 408288
len nodes: 31538

attribute edges: Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
attribute nodes: Index(['id', 'semantic_groups', 'preflabel', 'synonyms', 'name',
       'description'],
      dtype='object')


## Generate the NEO4J instance of the SR

In [9]:
%%time
# create a Neo4j server instance
neo4j_dir = neo4jlib.create_neo4j_instance('4.2.1')
print('The name of the neo4j directory is {}'.format(neo4j_dir))

# import to graph database
## prepare the graph to neo4j format
edges_df = utils.get_dataframe(edges)
nodes_df = utils.get_dataframe(nodes)
statements = neo4jlib.get_statements(edges_df)
concepts = neo4jlib.get_concepts(nodes_df)
print('statements: ', len(statements))
print('concepts: ',len(concepts))

## save files into neo4j import dir
neo4j_path = './{}'.format(neo4j_dir)
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type = 'statements')
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type = 'concepts')

## import graph into neo4j database
neo4jlib.do_import(neo4j_path)

Creating a Neo4j community v4.2.1 server instance...
Preparing the server...
Configuration adjusted!
Starting the server...
Neo4j v4.2.1 is running.
The name of the neo4j directory is neo4j-community-4.2.1
statements:  408288
concepts:  31538

File './neo4j-community-4.2.1/import/HD/HD_statements.csv' saved.

File './neo4j-community-4.2.1/import/HD/HD_concepts.csv' saved.

The function "do_import()" is running...

The graph is imported into the server. Neo4j is running.You can start exploring and querying for hypothesis. If you change ports or authentication in the Neo4j configuration file, the hypothesis-generation modules performance, hypothesis.py and summary.py, will be affected.

CPU times: user 3.41 s, sys: 164 ms, total: 3.57 s
Wall time: 24.9 s
